# Precise Toolpath

In [1]:
import rospy
from geometry_msgs.msg import PoseStamped
from moveit_msgs.msg import PlanningScene
from moveit_msgs.srv import GetPlanningScene
from moveit_commander import MoveGroupCommander
import numpy as np
import transforms3d as t3d
import open3d as o3d
from typing import List, Tuple
from moveit_msgs.msg import CollisionObject, PlanningScene, ObjectColor

from moveit_commander import PlanningSceneInterface

from geometry_msgs.msg import (
    Pose,
    PoseStamped,
    Point,
    Quaternion,
    Vector3,
)
from commander.msg import Goal
from commander.srv import (
    ExecuteTrajectory,
    PlanGoal,
    PlanGoalRequest,
    PlanSequence,
    PlanSequenceRequest,
    PickPlace,
    GetTcpPose,
    VisualizePoses,
    SetEe,
)

from commander.utils import poses_from_yaml, load_scene
from tf.transformations import euler_from_matrix, quaternion_from_euler
from shape_msgs.msg import Mesh
from geometry_msgs.msg import Point
from shape_msgs.msg import MeshTriangle
from std_msgs.msg import ColorRGBA
from visualization_msgs.msg import Marker
from geometry_msgs.msg import Pose
from shape_msgs.msg import SolidPrimitive
from moveit_msgs.msg import CollisionObject

decimal = 3
layers = 3
offset = 0.2
move_x = -0.7  #just for testing
move_z = 0.5  #just for testing

rospy.init_node("precise_toolpath")

load_scene()

plan_goal_srv = rospy.ServiceProxy("commander/plan_goal", PlanGoal)
plan_sequence_srv = rospy.ServiceProxy("commander/plan_sequence", PlanSequence)
execute_trajectory_srv = rospy.ServiceProxy("commander/execute_trajectory", ExecuteTrajectory)
get_tcp_pose_srv = rospy.ServiceProxy("commander/get_tcp_pose", GetTcpPose)
set_ee_srv = rospy.ServiceProxy("commander/set_ee", SetEe)
pick_place_srv = rospy.ServiceProxy("commander/pick_place", PickPlace)

def display_poses(poses: List[Pose], frame_id: str = "base_link") -> None:
    rospy.wait_for_service("/visualize_poses", timeout=10)
    visualize_poses = rospy.ServiceProxy("/visualize_poses", VisualizePoses)
    visualize_poses(frame_id, poses)

# def process_target_poses(target_pose):
   
#     # Plan the goal
#     success = plan_goal_srv(Goal(pose=target_pose, vel_scale=0.2, acc_scale=0.2, planner='ptp')).success

#     # Check if planning is successful
#     if success:
#         # Execute the trajectory
#         success = execute_trajectory_srv()

#         # Check if execution is successful
#         if not success:
#             rospy.loginfo("Failed to execute trajectory")

#     else:
#         rospy.loginfo("Failed to plan")


def create_obj(cube_pose, dimensions):

    scene_interface = PlanningSceneInterface()
    scene = PlanningScene()
    scene.is_diff = True


    # Create the cube collision object
    co = CollisionObject()
    co.header.frame_id = 'base_link'
    co.header.stamp = rospy.Time.now()
    co.id = 'cube'
    co.operation = CollisionObject.ADD
    
    # Define the cube as a solid primitive
    primitive = SolidPrimitive()
    primitive.type = SolidPrimitive.BOX
    primitive.dimensions = dimensions  # Width, Height, Depth
    
    # Define the pose of the cube
    pose = cube_pose
    pose.position.x = pose.position.x - 0.2
    pose.orientation = Quaternion(0.0, 0.0, 0.0, 1.0)
    
    # Attach the primitive and pose to the collision object
    co.primitives.append(primitive)
    co.primitive_poses.append(pose)
    scene.world.collision_objects.append(co)
    
    # Add object color (optional)
    oc = ObjectColor()
    oc.id = 'cube'
    oc.color = ColorRGBA(0.0, 1.0, 0.0, 0.7)  # Green color
    scene.object_colors.append(oc)

    scene_interface.apply_planning_scene(scene)

    rospy.sleep(1)  # Wait for the scene to be updated


def get_poses(cube_pose, dimensions):
    # Extracting cube pose information
    x, y, z = cube_pose.position.x, cube_pose.position.y, cube_pose.position.z
    roll, pitch, yaw = 0, 0, 0  # Assuming no rotation
    
    # Extracting dimensions information
    width, height, depth = dimensions
    
    # Calculating the half dimensions
    half_width = width / 2
    half_height = height / 2
    half_depth = depth / 2
    
    # Calculate relative positions of corners, face middle points, and edge middle points
    relative_positions = [
        np.array([half_width, half_height, half_depth]),   # Corner 1
        np.array([-half_width, half_height, half_depth]),  # Corner 2
        np.array([-half_width, -half_height, half_depth]), # Corner 3
        np.array([half_width, -half_height, half_depth]),  # Corner 4
        np.array([half_width, half_height, -half_depth]),   # Corner 5
        np.array([-half_width, half_height, -half_depth]),  # Corner 6
        np.array([-half_width, -half_height, -half_depth]), # Corner 7
        np.array([half_width, -half_height, -half_depth]),  # Corner 8
        np.array([0, 0, half_depth]),  # Front face middle
        np.array([0, 0, -half_depth]),  # Back face middle
        np.array([half_width, 0, 0]),  # Right face middle
        np.array([-half_width, 0, 0]),  # Left face middle
        np.array([0, half_height, 0]),  # Top face middle
        np.array([0, -half_height, 0]),  # Bottom face middle
        np.array([half_width, 0, half_depth]),   # Edge 1-2
        np.array([0, half_height, half_depth]),   # Edge 2-3
        np.array([-half_width, 0, half_depth]),  # Edge 3-4
        np.array([0, -half_height, half_depth]),  # Edge 1-4
        np.array([half_width, half_height, 0]),   # Edge 5-6
        np.array([-half_width, half_height, 0]),  # Edge 6-7
        np.array([-half_width, -half_height, 0]), # Edge 7-8
        np.array([half_width, -half_height, 0]),  # Edge 5-8
        np.array([0, half_height, -half_depth]),  # Edge 5-1
        np.array([0, half_height, half_depth]),   # Edge 6-2
        np.array([0, -half_height, half_depth]),  # Edge 7-3
        np.array([0, -half_height, -half_depth])  # Edge 8-4
    ]
    
    # Rotation matrix based on the cube's orientation
    rotation_matrix = np.array([
        [np.cos(yaw), -np.sin(yaw), 0],
        [np.sin(yaw), np.cos(yaw), 0],
        [0, 0, 1]
    ])
    
    # Calculate positions in the global frame
    poses = []
    for relative_pos in relative_positions:
        rotated_pos = np.dot(rotation_matrix, relative_pos)
        pose = Pose()
        pose.position.x = rotated_pos[0] + x
        pose.position.y = rotated_pos[1] + y
        pose.position.z = rotated_pos[2] + z
        pose.orientation = cube_pose.orientation
        
        # Check if pose is already in poses
        if not any(np.allclose([pose.position.x, pose.position.y, pose.position.z], [p.position.x, p.position.y, p.position.z]) for p in poses):
            poses.append(pose)
    
    return poses



# Define the pose of the cube
cube_pose = Pose()
cube_pose.position.x = -1.0 #-0.7-0.2=-0.9m   -0.7
cube_pose.position.y = 0.0
cube_pose.position.z = 0.46 # top of piece (not bottom) 0.62
cube_pose.orientation = Quaternion(-0.5, -0.5, 0.5, 0.5)  # No rotation

dimensions = [0.0, 0.2, 0.1]  # Width, height, depth

# cube_pose = Pose()
# cube_pose.position.x = -0.7 #-0.7-0.2=-0.9m
# cube_pose.position.y = 0.0
# cube_pose.position.z = 0.70 # top of piece (not bottom)
# cube_pose.orientation = Quaternion(-0.5, -0.5, 0.5, 0.5)  # No rotation

# dimensions = [0.0, 0.2, 0.25]  # Width, height, depth

poses = get_poses(cube_pose, dimensions)

# poses[0].orientation = Quaternion(-0.35, -0.6, 0.6, 0.35)
# poses[1].orientation = Quaternion(-0.6, -0.35, 0.35, 0.6)
# poses[-2].orientation = Quaternion(-0.35, -0.6, 0.6, 0.35)
# poses[-1].orientation = Quaternion(-0.6, -0.35, 0.35, 0.6)

create_obj(cube_pose, dimensions)
display_poses(poses)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# the following args for planner are available:
# - 'ompl' will use the ompl planning pipeline with RTTConnect planner
# - 'ptp' will use the pilz planning pipeline with the PTP planner
# - 'lin' will use the pilz planning pipeline with the LIN planner
cam_home = [-3.10042206640549, -2.131986729448063, 2.585211359700371, -3.5758736772415944, -1.5982155978220582, 0.0014838572949018819]
plan_goal_srv(Goal(joint_values=cam_home, vel_scale=0.2, acc_scale=0.2, planner='ptp'))
success = execute_trajectory_srv()

In [4]:
from sensor_msgs.msg import JointState
import yaml
import os

success = set_ee_srv('rgb_camera_tcp')

# Initialize array to store joint positions
joint_positions = []

def process_target_poses(target_pose):

    global joint_positions
   
    # Plan the goal
    success = plan_goal_srv(Goal(pose=target_pose, vel_scale=0.2, acc_scale=0.2, planner='ptp')).success

    # Check if planning is successful
    if success:
        # Execute the trajectory
        success = execute_trajectory_srv()

        # Check if execution is successful
        if not success:
            rospy.loginfo("Failed to execute trajectory")

    else:
        rospy.loginfo("Failed to plan")
    
    joint_states_msg = rospy.wait_for_message('/joint_states', JointState, timeout=5)
    joint_positions.append(joint_states_msg.position)
    

# Define a callback function to store joint positions
def joint_states_callback(msg):
    global joint_positions
    joint_positions.append(msg.position)

# Assuming you have a loop to plan and execute for each pose
for pose in poses:
    process_target_poses(pose)

# Define a custom YAML dumper to handle tuples without the python/tuple tag
class NoAliasDumper(yaml.SafeDumper):
    def represent_tuple(self, data):
        return self.represent_list(data)

# Get the current working directory
#current_directory = os.getcwd()

# Specify the YAML filename (joint_positions.yaml) in the current directory
#yaml_filename = os.path.join(current_directory, 'joint_positions.yaml')
yaml_filename = "/dev_ws/src/ur10e_examples/config/joint_positions.yaml"

# Save the joint positions array as YAML file using the custom dumper
with open(yaml_filename, 'w') as yaml_file:
    yaml.dump({'joint_positions': joint_positions}, yaml_file, Dumper=NoAliasDumper, default_flow_style=False)

In [5]:
import os
import yaml

#current_directory = os.getcwd()

# Read joint positions from the YAML file
#yaml_filename = os.path.join(current_directory, 'joint_positions.yaml')
yaml_filename = "/dev_ws/src/ur10e_examples/config/joint_positions.yaml"

try:
    with open(yaml_filename, 'r') as yaml_file:
        joint_positions_data = yaml.safe_load(yaml_file)
        joint_positions = joint_positions_data.get('joint_positions', [])
except FileNotFoundError:
    rospy.loginfo(f"YAML file '{yaml_filename}' not found. Please run the code that saves joint positions first.")
    exit()


def execute_joint_states(joint_position):
    # Plan the goal using the stored joint position
    success = plan_goal_srv(Goal(joint_values=joint_position, vel_scale=0.2, acc_scale=0.2, planner='ptp')).success

    # Check if planning is successful
    if success:
        # Execute the trajectory
        success = execute_trajectory_srv()

        # Check if execution is successful
        if not success:
            rospy.loginfo("Failed to execute trajectory")
            exit()
    else:
        rospy.loginfo("Failed to plan")
        exit()

for joint_position in joint_positions:
    execute_joint_states(joint_position)

In [6]:
# import rospy
# import moveit_commander

# def main():
#     # Initialize the MoveGroupCommander
#     robot = moveit_commander.RobotCommander()
#     group_name = "manipulator"  # replace this with your robot's arm group name
#     move_group = moveit_commander.MoveGroupCommander(group_name)
    
#     # Get the current pose of the end-effector
#     current_pose = move_group.get_current_pose().pose
    
#     # Print the current pose
#     print("Current Pose:")
#     print(current_pose)

# if __name__ == "__main__":
#     main()
